<a href="https://colab.research.google.com/github/bnaveenbharathi/DeepFake_Detection/blob/main/Deep_Fake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/deepfake/train'
val_dir = '/content/drive/MyDrive/deepfake/validation'
test_dir = '/content/drive/MyDrive/deepfake/test/images'


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# For validation and testing, only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary'
)

val_data = test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary'
)


Found 16 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [8]:
import tensorflow as tf

# Build a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Outputs 0-1
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save the model
model.save('/content/drive/My Drive/deepfake/deepfake_model.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.5000 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 13.6142
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.5000 - loss: 8.5908 - val_accuracy: 0.5000 - val_loss: 1.0356
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.8125 - loss: 0.9071 - val_accuracy: 0.5000 - val_loss: 1.4359
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.6250 - loss: 0.8371 - val_accuracy: 0.5000 - val_loss: 2.6181
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 1.4024 - val_accuracy: 0.5000 - val_loss: 1.8852
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.5000 - loss: 1.3282 - val_accuracy: 0.7500 - val_loss: 0.5406
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.7500 - loss: 0.5723 - val_accuracy: 0.5000 - val_loss: 1.1276
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.6875 - loss: 0.5959 - val_accuracy: 0.5000 - val_loss: 1.3069
Epoch 9/10
1/1 ━━━━━

In [10]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the model
model = tf.keras.models.load_model('/content/drive/My Drive/deepfake/deepfake_model.h5')

# Predict labels for test images
test_images_dir = test_dir
solution_file = '/content/drive/My Drive/deepfake/solution.csv'

with open(solution_file, 'w') as f:
    f.write('image_id,label\n')  # Header for submission

    for img_name in os.listdir(test_images_dir):
        img_path = os.path.join(test_images_dir, img_name)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Get prediction
        prediction = model.predict(img_array)[0][0]  # Single output
        f.write(f"{img_name},{prediction:.4f}\n")

print(f"Solution file saved to: {solution_file}")


Solution file saved to: /content/drive/My Drive/deepfake/solution.csv
